<a href="https://colab.research.google.com/github/aparnashenoy/ml/blob/master/xgboost_withoutsmote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
path = '/gdrive/My Drive/'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 20, 14

In [5]:
df = pd.read_csv(path + 'data_cvs_resp.csv')

In [6]:
df.drop(['Unnamed: 0'], axis = 1, inplace= True)

In [7]:
df.columns = map(str.lower, df.columns)
df.columns

Index(['hr', 'o2sat', 'sbp', 'map', 'dbp', 'resp', 'age', 'gender',
       'sepsislabel'],
      dtype='object')

In [8]:
df.head()

,hr,o2sat,sbp,map,dbp,resp,age,gender,sepsislabel
0,99.0,100.0,120.0,71.0,61.0,13.5,69.66,0,0
1,97.5,100.0,120.0,79.0,61.0,15.0,69.66,0,0
2,96.0,100.0,120.0,70.0,61.0,13.0,69.66,0,0
3,100.0,99.0,120.0,75.0,61.0,14.0,69.66,0,0
4,102.0,100.0,120.0,74.0,61.0,17.0,69.66,0,0


In [9]:
df=df.sample(n=20000, random_state=1234, axis=None, weights=None)

In [10]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [11]:
df['sepsislabel'] = df['sepsislabel'].astype('category')

In [12]:
df['gender']=df['gender'].astype('category')

In [13]:
y = df.sepsislabel.values
x = df.drop(["sepsislabel", 'gender'], axis = 1)

In [14]:
x=(x-x.mean())/(x.max()-x.min())

In [15]:
x.head()

,hr,o2sat,sbp,map,dbp,resp,age
350586,0.185139,0.079287,-0.258963,-0.061076,-0.021065,-0.040390,-0.324787
300080,-0.029147,-0.420713,0.544775,0.514681,0.520602,-0.123723,0.102475
1136672,0.518472,0.079287,0.096177,0.484378,0.520602,-0.040390,-0.159906
31178,-0.243433,0.412621,-0.455225,-0.313500,-0.021065,-0.332056,-0.062644
1135430,-0.052957,0.079287,0.395242,0.151045,-0.479398,-0.457056,-0.207525


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [17]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((14000, 7), (6000, 7), (14000,), (6000,))

In [18]:
# define model
model = XGBClassifier()
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(x_train, y_train)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.632636 using {'scale_pos_weight': 1}
0.632636 (0.052355) with: {'scale_pos_weight': 1}
0.630513 (0.055766) with: {'scale_pos_weight': 10}
0.620876 (0.057108) with: {'scale_pos_weight': 25}
0.614880 (0.056651) with: {'scale_pos_weight': 50}
0.610868 (0.056893) with: {'scale_pos_weight': 75}
0.610893 (0.055779) with: {'scale_pos_weight': 99}
0.605345 (0.059229) with: {'scale_pos_weight': 100}
0.588598 (0.053708) with: {'scale_pos_weight': 1000}
